```json
{
    "mask_name": "mask_00123.npy",
    "mask_height": 672,
    "mask_width": 630,
    "promote_type": "mask",
    "labels": {
        "3": {
            "instance_id": 3,
            "class_name": "table",
            "x1": 156,
            "y1": 333,
            "x2": 299,
            "y2": 414,
            "logit": 0.0
        },
        "5": {
            "instance_id": 5,
            "class_name": "table",
            "x1": 268,
            "y1": 463,
            "x2": 447,
            "y2": 666,
            "logit": 0.0
        }
    }
}
```

In [5]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

def analyze_npy_files(directory):
    directory = Path(directory)
    npy_files = list(directory.glob("*.npy"))
    total_nonzero_count = 0
    
    for npy_file in npy_files:
        mask = np.load(npy_file)
        
        # Get unique values and their counts
        unique_values, counts = np.unique(mask, return_counts=True)
        
        # Sum all values except 0
        total_nonzero_count += sum(counts[unique_values != 0])
        
        # Display histogram of values with integer bins
        plt.figure()
        plt.hist(mask.flatten(), bins=np.arange(min(unique_values), max(unique_values) + 2) - 0.5, edgecolor='black')
        plt.xticks(unique_values)  # Ensure only integer ticks
        plt.title(f"Value Distribution in {npy_file.name}")
        plt.xlabel("Mask Value")
        plt.ylabel("Count")
        plt.show()
    
    print(f"Total count of all values except 0: {total_nonzero_count}")
    return total_nonzero_count

# Example usage
directory = Path("path_to_npy_files")
total_nonzero_count = analyze_npy_files(directory)


Total count of all values except 0: 0


In [ ]:
analyze_npy_files("/tmp/mask_data")